# Reinforcement Learning (RL)

## Playing Atari with Deep RL [[Paper](https://arxiv.org/abs/1312.5602)]


## Installs


In [1]:
!pip install gym == 0.26.1
!pip install gym[atari]
!pip install autorom[accept-rom-license]


## Imports


In [14]:
import random
from collections import deque

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.tensorboard as tb
import tensorboard

import gym
from gym import spaces, wrappers

try:
    from tqdm.auto import trange
except Exception:
    trange = range

print(f"gym=={gym.__version__}")
print(f"numpy=={np.__version__}")
print(f"torch=={torch.__version__}")
print(f"tensorboard=={tensorboard.__version__}")


gym==0.26.1
numpy==1.23.3
torch==1.12.1+cpu
tensorboard==1.15.0


## Wrapper


In [18]:
cv2.ocl.setUseOpenCL(False)


class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == "NOOP"

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.integers(
                1, self.noop_max + 1
            )  # pylint: disable=E1101
        assert noops > 0
        obs, info = None, None
        for _ in range(noops):
            obs, _, done, _, info = self.env.step(self.noop_action)
            if done:
                obs, info = self.env.reset(**kwargs)
        return obs, info

    def step(self, ac):
        return self.env.step(ac)


class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == "FIRE"
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _, info = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _, info = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs, info

    def step(self, ac):
        return self.env.step(ac)


class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done = True

    def step(self, action):
        obs, reward, done, truncated, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, truncated, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs, info = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _, info = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs, info


class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros(
            (2,) + env.observation_space.shape, dtype=np.uint8)
        self._skip = skip

    def reset(self):
        return self.env.reset()

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, truncated, info = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, truncated, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84 as done in the Nature paper and later work.
        Expects inputs to be of shape height x width x num_channels
        """
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(
            low=0, high=255, shape=(self.height, self.width, 1), dtype=np.uint8
        )

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(
            frame, (self.width, self.height), interpolation=cv2.INTER_AREA
        )
        return frame[:, :, None]


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        Expects inputs to be of shape num_channels x height x width.
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(
            low=0, high=255, shape=(shp[0] * k, shp[1], shp[2]), dtype=np.uint8
        )

    def reset(self):
        ob, _ = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, trunc, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, trunc, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))


class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0


class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers."""
        self._frames = frames

    def __array__(self, dtype=None):
        out = np.concatenate(self._frames, axis=0)
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._frames)

    def __getitem__(self, i):
        return self._frames[i]


class PyTorchFrame(gym.ObservationWrapper):
    """Image shape to num_channels x height x width"""

    def __init__(self, env):
        super(PyTorchFrame, self).__init__(env)
        shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0.0, high=1.0, shape=(shape[-1], shape[0], shape[1]), dtype=np.uint8
        )

    def observation(self, observation):
        return np.rollaxis(observation, 2)


## Environmnet


In [6]:
def trigger(x):
    if x % 100 == 0:
        print(x)
    return x > hyper_params['num_steps'] - 600


class TrainingEnvironment:
    def __init__(self, env_name, seed):       
        assert "NoFrameskip" in env_name, "Require environment with no frameskip"

        self.seed = seed

        self.env = gym.make(env_name, render_mode="rgb_array")
        self.env.seed(seed)

        self.env = NoopResetEnv(self.env, noop_max=30)
        self.env = MaxAndSkipEnv(self.env, skip=4)
        self.env = EpisodicLifeEnv(self.env)
        self.env = FireResetEnv(self.env)
        self.env = ClipRewardEnv(self.env)
        self.env = WarpFrame(self.env)
        self.env = PyTorchFrame(self.env)
        self.env = wrappers.RecordVideo(self.env, "videos")

    def unwrap(self):
        return self.env

    def __str__(self):
        return f"seed = {self.seed}\nstates = {self.env.observation_space.shape}\nactions = {self.env.action_space.n}"


## Replay Buffer


In [7]:
class ReplayBuffer:
    """
    Simple storage for transitions from an environment.
    """

    def __init__(self, env, size, device):
        self._maxsize = size
        self._mem_counter = 0
        self._state_shape = env.observation_space.shape
        self._num_actions = env.action_space.n
        self._device = device

        self.states = torch.zeros(self._maxsize,
                                  *self._state_shape,
                                  dtype=torch.float32).to(self._device)
        self.actions = torch.zeros(self._maxsize,
                                   dtype=torch.int64).to(self._device)
        self.rewards = torch.zeros(self._maxsize,
                                   dtype=torch.float32).to(self._device)
        self.next_states = torch.zeros(self._maxsize,
                                       *self._state_shape,
                                       dtype=torch.float32).to(self._device)
        self.dones = torch.zeros(self._maxsize,
                                 dtype=torch.float32).to(self._device)

    def __call__(self, indices):
        return self.states[indices],\
            self.actions[indices],\
            self.rewards[indices],\
            self.next_states[indices],\
            self.dones[indices]

    def state_dict(self):
        dict = {
            'maxsize': self._maxsize,
            'mem_counter': self._mem_counter,
            'state_shape': self._state_shape,
            'num_actions': self._num_actions,
            'states': self.states.detach(),
            'actions': self.actions.detach(),
            'rewards': self.rewards.detach(),
            'next_state': self.next_states.detach(),
            'dones': self.dones.detach(),
        }

        return dict

    def load_state_dict(self, state_dict):
        self._maxsize = state_dict['maxsize']
        self._mem_counter = state_dict['mem_counter']
        self._state_shape = state_dict['state_shape']
        self._num_actions = state_dict['num_actions']

        self.states = torch.load(state_dict['states'],
                                 map_location=self._device)
        self.actions = torch.load(state_dict['actions'],
                                  map_location=self._device)
        self.rewards = torch.load(state_dict['rewards'],
                                  map_location=self._device)
        self.next_states = torch.load(state_dict['next_states'],
                                      map_location=self._device)
        self.dones = torch.load(state_dict['dones'],
                                map_location=self._device)

    def add(self, state, action, reward, next_state, done):
        """
        Add a transition to the buffer. Old transitions will be overwritten if the buffer is full.
        :param state: the agent's initial state
        :param action: the action taken by the agent
        :param reward: the reward the agent received
        :param next_state: the subsequent state
        :param done: whether the episode terminated
        """
        next_index = self._mem_counter % self._maxsize

        self.states[next_index] = torch.from_numpy(state)
        self.actions[next_index] = action
        self.rewards[next_index] = reward
        self.next_states[next_index] = torch.from_numpy(next_state)
        self.dones[next_index] = float(done)

        self._mem_counter += 1

    def last_frames(self, n):
        """
        Get last n frames in the memory buffer
        :param n: the number of frames
        :return: a batch of n frames
        """

        memory = min(self._mem_counter, self._maxsize) - 1
        assert self._mem_counter >= n, f"memory={memory} must have at least n={n} frames"

        index = self._mem_counter % self._maxsize
        indices = range(index-n, index)

        return self(indices)

    def sample(self, batch_size):
        """
        Randomly sample a batch of transitions from the buffer.
        :param batch_size: the number of transitions to sample
        :return: a mini-batch of sampled transitions
        """
        memory = min(self._mem_counter, self._maxsize) - 1
        assert self._mem_counter >= batch_size, f"memory={memory} must have at least batch_size={batch_size} frames"

        indices = np.random.randint(0, memory, size=batch_size)

        return self(indices)


## Agent


In [11]:
class DQNAgent:
    def __init__(self, env, memory_size, use_double_dqn, lr, batch_size, gamma, device, log_dir):
        self.num_actions = env.action_space.n
        self.batch_size = batch_size
        self.gamma = gamma
        self.device = device
        self.use_double_dqn = use_double_dqn
        self.episode_rewards = [0.0]

        # Tensorboard
        self.tb_w = tb.SummaryWriter(log_dir)

        # Q-networks
        self.Q = self._build_model().to(self.device)
        self.Q_target = self._build_model().to(self.device)

        # Loss
        self.loss = nn.MSELoss()

        # Adam optimizer
        self.optimizer = torch.optim.Adam(self.Q.parameters(), lr=lr)

        # Replay buffer/memory
        self.memory = ReplayBuffer(env=env, size=memory_size, device=device)

        # Indexing
        self.idx = 0

    def _build_model(self):
        # 1 channel -> 16 features -> 32 features -> 32*9*9=2592 features -> 256 -> # actions
        return nn.Sequential(nn.Conv2d(1, 16, kernel_size=8, stride=4, padding=0),
                             nn.ReLU(),
                             nn.Conv2d(16, 32, kernel_size=4,
                                       stride=2, padding=0),
                             nn.ReLU(),
                             nn.Flatten(),
                             nn.Linear(32*9*9, 256), nn.ReLU(),
                             nn.Linear(256, self.num_actions))

    def optimise_td_loss(self):
        """
        Optimise the TD-error over a single minibatch of transitions
        :return: the loss
        """
        states, actions, rewards, next_states, dones = \
            self.memory.sample(self.batch_size)

        # Normalize memory states
        states = states / 255
        next_states = next_states / 255

        # Targets
        with torch.no_grad():  # Not used in gradient calculation
            if self.use_double_dqn:
                target_actions = target_values = self.Q(next_states)\
                    .argmax(dim=1, keepdim=True)

                target_values = self.Q_target(next_states)\
                    .gather(dim=1, index=target_actions).flatten()

            else:
                # torch.max() -> Tensor[values], Tensor[indices]
                target_values = self.Q_target(next_states)\
                    .max(dim=1, keepdim=True)[0].flatten()

            targets = rewards + \
                self.gamma * target_values * (1 - dones)

        # Online
        values = self.Q(states)\
            .gather(dim=1, index=actions.unsqueeze(-1))\
            .flatten()

        # Loss
        loss = self.loss(values, targets).to(self.device)

        # Gradient descent
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Log to tensorboard
        self.tb_w.add_scalar("Loss", loss.item(), self.idx)
        for name, weight in self.Q.named_parameters():
            self.tb_w.add_histogram('Q-%s' % name, weight, self.idx)
            self.tb_w.add_histogram('Q-%s.grad' % name, weight, self.idx)

        for name, weight in self.Q_target.named_parameters():
            self.tb_w.add_histogram('Q-target-%s' % name, weight, self.idx)
            self.tb_w.add_histogram('Q-target-%s' % name, weight, self.idx)

        self.idx += 1

    def update_target_network(self):
        """
        Update the target Q-network by copying the weights from the current Q-network
        """
        self.Q_target.load_state_dict(self.Q.state_dict())

    def remember(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)

        # Rewards
        self.episode_rewards[-1] += reward

        if done:
            self.episode_rewards.append(0.0)

    def num_episodes(self):
        return len(self.episode_rewards)

    def act(self, state: torch.Tensor):
        """
        Select an action greedily from the Q-network given the state
        :param state: the current state
        :return: the action to take
        """
        # Last four frames (states)
        batch, _, _, _, _ = self.memory.last_frames(4)

        # Replace first frame (state)
        batch[0, :, :, :] = state

        # Normalize memory states
        batch = batch / 255

        with torch.no_grad():
            # Greedy action (max)
            return self.Q(batch).argmax(dim=1, keepdim=True)[1].view(1, 1)

    def log(self, t):
        self.tb_w.add_scalar("Episodes", self.num_episodes(), t)

        mean_100ep_reward = round(np.mean(self.episode_rewards[-101:-1]), 1)
        self.tb_w.add_scalar("Mean Reward (100 episodes)",
                             mean_100ep_reward, t)

    def save(self, path):
        print('saving models...')

        checkpoint = {
            'Q': self.Q.state_dict(),
            'Q_target': self.Q_target.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'loss': self.loss,
            'memory': self.memory.state_dict()
        }

        torch.save(checkpoint, path)

    def load(self, path):
        print('loading models...')

        checkpoint = torch.load(path)

        self.Q.load_state_dict(checkpoint.get('Q'), map_location=self.device)
        self.Q.train()

        self.Q_target.load_state_dict(checkpoint.get(
            'Q_target'), map_location=self.device)
        self.Q_target.train()

        self.optimizer.load_state_dict(checkpoint.get('optimizer'))

        self.loss = checkpoint.get('loss')

        self.memory.load_state_dict(checkpoint.get('memory'))


## Hyperparameters

In [ ]:
hyper_params = {
    "seed": 42,
    "env": "PongNoFrameskip-v4",
    "use-double-dqn": True,
    "replay-buffer-size": int(5e3),
    "batch-size": 256,
    "num-steps": int(1e6),
    "learning-starts": int(1e4),
    "learning-freq": 5,
    "target-update-freq": int(1e3),
    "learning-rate": int(1e-4),
    "discount-factor": 0.99,
    "eps-start": 1.0,
    "eps-end": 0.01,
    "eps-fraction": 0.1,
}

## Training Loop


In [9]:
args_vars = {
    'device': "cuda" if torch.cuda.is_available() else "cpu",
    'log_dir': 'runs',
    'stat_freq': 20,
    'out': 'data/model.pth'
}

def get_epsilon_threshold(t):
    diff = hyper_params['eps-end'] - hyper_params['eps-start']
    fraction = float(t) / hyper_params['eps-fraction'] * \
        float(hyper_params['num-steps'])
    fraction = min(1.0, fraction)
    return hyper_params['eps-start'] + fraction * diff

if __name__ == '__main__':
    device = torch.device(args_vars.get("device"))
    print("device = %s" % device)

    np.random.seed(hyper_params['seed'])
    random.seed(hyper_params['seed'])

    train_env = TrainingEnvironment(env_name=hyper_params['env'],
                                    seed=hyper_params['seed'],
                                    steps=hyper_params['num-steps'])

    env = train_env.unwrap()

    agent = DQNAgent(env=env,
                        memory_size=hyper_params['replay-buffer-size'],
                        use_double_dqn=hyper_params['use-double-dqn'],
                        lr=hyper_params['learning-rate'],
                        batch_size=hyper_params['batch-size'],
                        gamma=hyper_params['discount-factor'],
                        device=device,
                        log_dir=args_vars['log_dir'])

    in_file = args_vars.get('in')
    if in_file is not None:
        try:
            agent.load(in_file)
        except FileNotFoundError:
            print("model file not found: %s" % in_file)

    try:
        state, _ = env.reset()
        for t in trange(hyper_params['num-steps']):
            epsilon_threshold = get_epsilon_threshold(t, hyper_params)

            #  select random action if sample is less equal than eps_threshold
            if (t <= hyper_params['learning-starts'] or random.random() <= epsilon_threshold):
                action = env.action_space.sample()
            else:
                action = agent.act(torch.tensor(state))

            # take step in env
            next_state, reward, done, _, _ = env.step(action)

            # add state, action, reward, next_state, float(done) to reply memory - cast done to float
            agent.remember(state=state,
                            action=action,
                            reward=reward,
                            next_state=next_state,
                            done=float(done))

            # Update state
            state = next_state

            if done:
                state, _ = env.reset()

                if (agent.num_episodes() % args_vars.get('stat_freq') == 0):
                    explore_time = int(100 * epsilon_threshold)
                    agent.log(t)
                    agent.tb_w.add_scalar("Explore time", explore_time, t)

            if t > hyper_params['learning-starts']:
                if t % hyper_params['learning-freq'] == 0:
                    agent.optimise_td_loss()

                if t % hyper_params['target-update-freq'] == 0:
                    agent.update_target_network()

    finally:
        agent.tb_w.close()
        out_file = args_vars.get('out')
        if out_file is not None:
            agent.save(out_file)


device = cpu


c:\Users\LENOVO\.pyenv-win-venv\envs\rl_lab_3\lib\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at c:\Users\LENOVO\Develop\coms4061a-labs\labs\Lab 3\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


seed = 42
states = (1, 84, 84)
actions = 6


TypeError: DQNAgent.__init__() missing 1 required positional argument: 'tb_path'